# Setup

In [ ]:
!pip install datacleaner

In [ ]:
from IPython.display import display
import pandas as pd
import numpy as np
from datacleaner import autoclean

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Settings
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# data read encoding changed to the russian windows version.
gDrivePath = '/gdrive/MyDrive/Colab Notebooks/data/'
original_data = pd.read_csv(gDrivePath+'SS_data.csv', encoding='windows-1251')

In [ ]:
def col_to_numeric(col):
    try:
        return pd.to_numeric(col)
    except:
        return col

def binaryCol(col):
    try:
        return pd.get_dummies(col, drop_first=True) if col.nunique() == 2 else col
    except TypeError:
        return col
    
def extractYearFromDate(row):
    return int(row.split('/')[-1]) if type(row)==str else row

# Outputs the columns that need to be dropped
def to_drop(dataset, corr_threshold, useless_columns):
    # Create correlation matrix
    corr_matrix = dataset.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find index of feature columns with correlation greater than corr_threshold
    drop = [column for column in upper.columns if any(upper[column] > corr_threshold)]
    drop = drop + useless_columns
    return drop

def conditionalSplit(sent):
    if str(sent) != 'nan':
        return list(map(str.strip, str(sent).replace(',', '@').replace('&', '@').replace(' and ', '@').split('@')))
    return sent

In [ ]:
def guess_data_type(column):
    value_counts = column.value_counts()
    if len(value_counts) == 2:
        return 'Binary'
    elif (len(value_counts) / column.shape[0]) < 0.05:
        return 'Categorical'
    else:
        return 'Text'

def low_freq_combine(data, threshold, value='other'):
    high_freq = data.value_counts()[(data.value_counts() / data.notnull().sum()) >= threshold].index
    data = data.where((data.isin(high_freq) | data.isnull()), other=value)
    return data

# Dataset overview

In [ ]:
# Dataset preview
original_data.info()
original_data.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Columns: 116 entries, Company_Name to Renown score
dtypes: float64(5), int64(3), object(108)
memory usage: 427.9+ KB


,Company_Name,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Short Description of company profile,Industry of company,Focus functions of company,Investors,Employee Count,Employees count MoM change,Has the team size grown,Est. Founding Date,Last Funding Date,Last Funding Amount,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Presence of a top angel or venture fund in previous round of investment,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Specialization of highest education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Employee benefits and salary structures,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,Client Reputation,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Gartner hype cycle stage,Time to maturity of technology (in years),Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,Company1,Success,No Info,No Info,-1.0,Video di

In [ ]:
# Columns with unique values
original_data.nunique()[original_data.nunique() > len(original_data)/2]
# Company_Name could be an index or dropped

Company_Name                            472
Internet Activity Score                 260
Short Description of company profile    313
Investors                               319
Last Funding Date                       284
google page rank of company website     311
dtype: int64

# Data Cleanup Process

In [ ]:
# identifying dataset features
to_drop = ['Investors']
missingIdentifiers = ['n/a', 'nan', 'no info', 'unknown amount']

In [ ]:
# 1) Cleanup: lowercase objects, find NaNs, map ordinal values, find numeric columns
# 2) Cleanup: drop useless columns
data = original_data.apply(lambda col: col.str.strip().str.lower() if col.dtype == 'object' else col)
data = data.applymap(lambda val: np.NaN if str(val).strip().lower() in missingIdentifiers else val)
data = data.apply(col_to_numeric)
data = data.drop(to_drop, axis=1, errors='ignore')

# Datetime columns
data['Est. Founding Date'] = data['Est. Founding Date'].apply(extractYearFromDate)
data['Last Funding Date'] = data['Last Funding Date'].apply(extractYearFromDate)

# Special columns
data['Country of company'] = low_freq_combine(data['Country of company'], 0.01, 'other country')
data['Continent of company'] = low_freq_combine(data['Continent of company'], 0.01, 'other continent')

text_columns = ['Short Description of company profile', 'Industry of company', 
                'Focus functions of company', 'Specialization of highest education']
splitters = {'&': ' ', ',': ' ', 'and': ' ', '|':' '}

data['Short Description of company profile'].str.lower().fillna('') 
data['Industry of company'] = data['Industry of company'].str.lower().fillna('').apply(lambda row: row.replace('|', ' ') if str(row) != 'nan' else row)
data['Focus functions of company'] = data['Focus functions of company'].str.lower().fillna('').apply(lambda x: x.translate(splitters))
data['Specialization of highest education'] = data['Specialization of highest education'].str.lower().fillna('').apply(lambda x: x.translate(splitters))

data['Description'] = data[text_columns].fillna('').apply(' '.join, axis=1)

# Adding dummies to dataset
clean_dataset = data.drop(text_columns, axis=1)

In [ ]:
print('Shape:',clean_dataset.shape)
clean_dataset.head()

Shape: (472, 112)


,Company_Name,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Employee Count,Employees count MoM change,Has the team size grown,Est. Founding Date,Last Funding Date,Last Funding Amount,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Presence of a top angel or venture fund in previous round of investment,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Employee benefits and salary structures,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,Client Reputation,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Gartner hype cycle stage,Time to maturity of technology (in years),Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score,Description
0,company1,success,NaN,NaN,-1.0,3.0,0.0,no,NaN,2013.0,450000.0,united states,north america,2.0,0.0,1,2,2,15.0,yes,4.0,nothing,no,small,no,no,no,no,service,no,private,

In [ ]:
dataset = clean_dataset.set_index('Company_Name')
dataset = dataset.drop_duplicates()
display(dataset.head())
print(dataset.shape)

,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Employee Count,Employees count MoM change,Has the team size grown,Est. Founding Date,Last Funding Date,Last Funding Amount,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Presence of a top angel or venture fund in previous round of investment,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Employee benefits and salary structures,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,Client Reputation,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Gartner hype cycle stage,Time to maturity of technology (in years),Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score,Description
Company_Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
company1,success,NaN,NaN,-1.0,3.0,0.0,no,NaN,2013.0,450

(470, 111)


In [ ]:
dataset.to_csv(gDrivePath+'SS_dataset.csv')